In [17]:
import requests

response = requests.post("http://127.0.0.1:8000/predict", 
                         json={
                                "input": 4.0, 
                               "strategy": "kmeans", 
                               "pipeline": "csv", 
                               "file_path": "machine_learning/data/ALL_DATA.csv"
                               })
print(f"Status: {response.status_code}\nResponse:\n {response.text}")


Status: 200
Response:
 {"output":{"output":"K-Means result for [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
response

<Response [200]>

In [12]:
import json


result = json.loads(response.text)

In [21]:
import sys
import os
from sklearn.cluster import KMeans, DBSCAN
import litserve as ls
from abc import ABC, abstractmethod
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [22]:
os.chdir("../")

### Preprocessing Steps: Cleaning and Encoding Data

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('machine_learning/data/DataCoSupplyChainDataset.csv', delimiter=';', encoding='latin1')

# Drop columns with no meaningful data
data = data.drop(['Product Description', 'Customer Password', 'Customer Email', 'Product Image'], axis=1)

# Handle missing values
data['Order Zipcode'] = data['Order Zipcode'].fillna(-1)

# Convert date columns to datetime format
data['order_date'] = pd.to_datetime(data['order date (DateOrders)'])
data['shipping_date'] = pd.to_datetime(data['shipping date (DateOrders)'])

# Create new feature: Shipping Delay
data['shipping_delay'] = (data['shipping_date'] - data['order_date']).dt.days

# Extract time-based features
data['order_year'] = data['order_date'].dt.year
data['order_month'] = data['order_date'].dt.month
data['order_day'] = data['order_date'].dt.day

# Encode categorical columns
categorical_columns = ['Type', 'Delivery Status', 'Category Name', 'Customer City', 'Customer Country', 'Customer Segment', 'Market', 'Order Region', 'Order State', 'Order Status', 'Shipping Mode']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Display the first few rows of the cleaned dataset
data.head()

### Clustering or Segmentation

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Select numerical columns for clustering
numerical_columns = ['Days for shipping (real)', 'Days for shipment (scheduled)', 'Benefit per order', 'Sales per customer', 'shipping_delay']
X = data[numerical_columns]

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(X_scaled)

# Display the first few rows with cluster labels
data[['Cluster'] + numerical_columns].head()

In [31]:
data = pd.read_csv('data/ALL_DATA.csv')
label_encoder = LabelEncoder()
data['Company_Label'] = label_encoder.fit_transform(data['Ticker'])
if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'])
    data['Date'] = data['Date'].map(pd.Timestamp.toordinal)
data.head()

,Ticker,Date,Open,High,Low,Close,Volume,Company_Label
0,AB,738452,24.58,24.58,24.58,24.58,130,0
1,AB,738454,23.90,24.58,23.90,23.90,14301,0
2,AB,738455,24.58,24.58,24.57,24.58,505,0
3,AB,738456,24.58,24.58,24.09,24.58,5882,0
4,AB,738459,24.58,24.58,24.57,24.58,1240,0


In [32]:
data.shape

(187987, 8)

In [33]:
data.Company_Label.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87])

In [34]:
data = data.sample(frac=1).reset_index(drop=True)
data.shape

(187987, 8)

In [35]:
data.head()

,Ticker,Date,Open,High,Low,Close,Volume,Company_Label
0,MNP,735040,27.13,27.48,27.00,27.48,1180,38
1,CREAL,737692,7.20,7.20,7.20,7.20,1,26
2,UADH,737028,2.42,2.42,2.23,2.27,81890,82
3,AMS,736118,1.98,2.02,1.96,1.97,7961,6
4,TPR,737780,4.07,4.25,4.07,4.23,8531,79


In [37]:
data.drop(columns=['Ticker'], inplace=True)

In [40]:
data.dropna(inplace=True)

In [41]:
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit(data)

In [46]:
set(clusters.labels_.tolist())

{0, 1, 2}